# Scatterplot

In [21]:
import os
from bokeh.resources import CDN
from bokeh.embed import file_html

# Path to save the file
output_dir = "/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/thomasroennebek.github.io/assets/images/interactive"
output_file = os.path.join(output_dir, "scatterplot2-plot.html")

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Generate HTML and write to file
html = file_html(layout, CDN, "Interactive Crime Explorer")

with open(output_file, "w") as f:
    f.write(html)

In [ ]:
import pandas as pd
import numpy as np
from bokeh.io import output_file, save
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, Select, Div
from bokeh.layouts import column, row

########################
# 1) DATA PREPARATION
########################

# Load data
sf_crime_data = pd.read_csv(
    "/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/Data/Police_Incident_Reports_2003_2025_Cleaned.csv",
    low_memory=False
)

# Extract incident hour
sf_crime_data["Incident Hour"] = pd.to_datetime(
    sf_crime_data["Incident Time"], format="%H:%M", errors="coerce"
).dt.hour

# Map days to indices
day_to_index = {
    "Monday": 0, "Tuesday": 1, "Wednesday": 2,
    "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6
}
sf_crime_data["DayIndex"] = sf_crime_data["Incident Day of Week"].map(day_to_index)

# Filter for focus crimes
focuscrimes = [
    "Weapon Laws", "Prostitution", "Robbery", "Burglary", "Assault",
    "Drug/Narcotic", "Larceny Theft", "Vandalism", "Vehicle Theft", "Stolen Property"
]
focus_crime_data = sf_crime_data[
    sf_crime_data["Incident Category"].isin(focuscrimes) &
    sf_crime_data["Incident Hour"].notnull() &
    sf_crime_data["DayIndex"].notnull()
].copy()

# Calculate HourOfWeek
focus_crime_data["HourOfWeek"] = focus_crime_data["DayIndex"] * 24 + focus_crime_data["Incident Hour"]
hourly_counts = focus_crime_data.groupby(["Incident Category", "HourOfWeek"]).size().unstack(fill_value=0)

########################
# 2) BOKEH INTERACTIVE SCATTER
########################

# Prepare data for plotting
crime_list = sorted(hourly_counts.index)
all_hours = np.arange(168)

# Build unfiltered data dictionary
unfiltered_data = {"hour_of_week": all_hours.tolist()}
for crime in crime_list:
    unfiltered_data[crime] = hourly_counts.loc[crime].values.astype(int).tolist()

# Initial crimes for comparison
init_crime1 = crime_list[0]
init_crime2 = crime_list[1]

# Initial data for the plot
x_init = hourly_counts.loc[init_crime1].values.astype(int)
y_init = hourly_counts.loc[init_crime2].values.astype(int)
h_init = all_hours

# Create ColumnDataSource
source = ColumnDataSource(data=dict(hour=h_init, x=x_init, y=y_init))

# Create figure
p = figure(
    title="Compare Weekly Crime Patterns",
    x_axis_label="Crime 1",
    y_axis_label="Crime 2",
    width=700, height=400,
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# Dark styling for the plot
p.background_fill_color = "#252a34"
p.border_fill_color = "#252a34"
p.outline_line_color = None
p.title.text_color = "#eaeaea"
p.axis.axis_line_color = "#eaeaea"
p.axis.major_label_text_color = "#eaeaea"
p.axis.major_tick_line_color = "#eaeaea"
p.axis.minor_tick_line_color = "#eaeaea"
p.xaxis.axis_label_text_color = "#eaeaea"
p.yaxis.axis_label_text_color = "#eaeaea"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add scatter plot
scatter = p.scatter(
    x="x", y="y", source=source,
    size=7, color="#00adb5", alpha=0.8, line_color=None
)

# Add hover tool
hover = HoverTool(tooltips=[
    ("HourOfWeek", "@hour"),
    ("Crime 1", init_crime1),
    ("Crime 2", init_crime2)
])
p.add_tools(hover)

# Create Select widgets
select_crime1 = Select(title="Select Crime 1:", value=init_crime1, options=crime_list)
select_crime2 = Select(title="Select Crime 2:", value=init_crime2, options=crime_list)

# Define custom CSS to style Select widget titles
custom_css = """
<style>
    .bk-root .bk-input-group > label {
        color: white !important;
    }
</style>
"""
div = Div(text=custom_css)

# Callback to update data based on selections
callback_code = """
var c1 = select_crime1.value;
var c2 = select_crime2.value;

var hoursAll = unfiltered_data['hour_of_week'];
var arrX = unfiltered_data[c1];
var arrY = unfiltered_data[c2];

var new_hour = [];
var new_x = [];
var new_y = [];

for (var i = 0; i < hoursAll.length; i++) {
    new_hour.push(hoursAll[i]);
    new_x.push(arrX[i]);
    new_y.push(arrY[i]);
}

source.data['hour'] = new_hour;
source.data['x'] = new_x;
source.data['y'] = new_y;
source.change.emit();

// Update hover tool
hover.tooltips = [
    ["HourOfWeek", "@hour"],
    ["Crime 1", c1],
    ["Crime 2", c2]
];
"""

callback = CustomJS(
    args=dict(
        source=source,
        unfiltered_data=unfiltered_data,
        select_crime1=select_crime1,
        select_crime2=select_crime2,
        hover=hover
    ),
    code=callback_code
)

select_crime1.js_on_change("value", callback)
select_crime2.js_on_change("value", callback)

# Arrange layout
layout = column(div, row(select_crime1, select_crime2), p)

# Specify the output file path
output_file("/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/thomasroennebek.github.io/assets/images/scatterplot.html")

# Save the plot as an HTML file
save(layout)

'/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/thomasroennebek.github.io/assets/images/scatterplot.html'

In [24]:
sf_crime_data.head(50)

,Incident Date,Incident Time,Incident Day of Week,Incident ID,Incident Number,Incident Code,Incident Category,Incident Description,Resolution,Police District,Analysis Neighborhood,Latitude,Longitude,Point,Incident Hour,DayIndex
0,2023-03-01,05:02,Wednesday,1253795,236046151,6374,Larceny Theft,"theft, other property, >$950",open or active,mission,NaN,NaN,NaN,NaN,5,2
1,2023-03-13,10:59,Monday,1253551,230174885,7041,Recovered Vehicle,"vehicle, recovered, auto",open or active,out of sf,NaN,NaN,NaN,NaN,10,0
2,2023-03-14,18:44,Tuesday,1254024,230176728,7041,Recovered Vehicle,"vehicle, recovered, auto",open or active,out of sf,NaN,NaN,NaN,NaN,18,1
3,2023-02-15,03:00,Wednesday,1253786,236046123,6372,Larceny Theft,"theft, other property, $50-$200",open or active,mission,NaN,NaN,NaN,NaN,3,2
4,2023-03-13,11:26,Monday,1254195,236046850,6244,Larceny Theft,"theft, from locked vehicle, >$950",open or active,central,NaN,NaN,NaN,NaN,11,0
5,2023-03-11,15:00,Saturday,1254206,236045937,6244,Larceny Theft,"theft, from locked vehicle, >$950",open or active,central,NaN,NaN,NaN,NaN,15,5
6,2023-03-13,07:30,Monday,1254123,236047096,6244,Larceny Theft,"theft, from locked vehicle, >$950",open or active,central,NaN,NaN,NaN,NaN,7,0
7,2022-06-27,12:00,Monday,1254393,230184129,71000,Lost Property,lost property,open or active,central,Financial District/South Beach,37.787359,-122.408227,POINT (-122.40822672700406 37.78735926098589),12,0
8,2023-03-16,09:26,Thursday,1254679,230185672,16780,Drug Violation,"firearm, armed while possessing controlled sub...",cite or arrest adult,out of sf,NaN,NaN,NaN,NaN,9,3
9,2023-03-16,17:30,Thursday,1254826,230187101,4134,Assault,battery,open or active,bayview,Potrero Hill,37.762290,-122.401324,POINT (-122.40132418490647 37.76228996810526),17,3


### This works

In [22]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, output_file, save
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, Select
from bokeh.layouts import column, row
import json

########################
# 1) DATA PREPARATION
########################

# Load data
sf_crime_data = pd.read_csv(
    "/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/Data/Police_Incident_Reports_2003_2025_Cleaned.csv",
    low_memory=False
)

# Extract incident hour
sf_crime_data["Incident Hour"] = pd.to_datetime(
    sf_crime_data["Incident Time"], format="%H:%M", errors="coerce"
).dt.hour

# Map days to indices
day_to_index = {
    "Monday": 0, "Tuesday": 1, "Wednesday": 2,
    "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6
}
sf_crime_data["DayIndex"] = sf_crime_data["Incident Day of Week"].map(day_to_index)

# Filter relevant crimes
focuscrimes = [
    "Weapon Laws", "Prostitution", "Robbery", "Burglary", "Assault",
    "Drug/Narcotic", "Larceny Theft", "Vandalism", "Vehicle Theft", "Stolen Property"
]
focus_crime_data = sf_crime_data[
    sf_crime_data["Incident Category"].isin(focuscrimes) &
    sf_crime_data["Incident Hour"].notnull() &
    sf_crime_data["DayIndex"].notnull()
].copy()

focus_crime_data["HourOfWeek"] = focus_crime_data["DayIndex"] * 24 + focus_crime_data["Incident Hour"]
hourly_counts = focus_crime_data.groupby(["Incident Category", "HourOfWeek"]).size().unstack(fill_value=0)

########################
# 2) BOKEH INTERACTIVE SCATTER
########################

output_notebook()

crime_list = sorted(hourly_counts.index)
all_hours = np.arange(168)

# Create mapping from hour to (day, time)
hour_to_day_and_time = {
    int(hour): (
        (pd.Timestamp("2023-01-02") + pd.Timedelta(hours=int(hour))).strftime("%A"),
        (pd.Timestamp("2023-01-02") + pd.Timedelta(hours=int(hour))).strftime("%H:%M")
    )
    for hour in all_hours
}

# Unfiltered data dictionary
unfiltered_data = {"hour_of_week": all_hours.tolist()}
for crime in crime_list:
    unfiltered_data[crime] = hourly_counts.loc[crime].values.astype(int).tolist()

init_crime1 = crime_list[0]
init_crime2 = crime_list[1]

x_init = hourly_counts.loc[init_crime1].values.astype(int)
y_init = hourly_counts.loc[init_crime2].values.astype(int)
h_init = all_hours
days = [hour_to_day_and_time[int(h)][0] for h in h_init]
times = [hour_to_day_and_time[int(h)][1] for h in h_init]

# ColumnDataSource with day/time
source = ColumnDataSource(data=dict(
    hour=h_init,
    x=x_init,
    y=y_init,
    day=days,
    time=times
))

# Create plot
p = figure(
    title="Compare Weekly Crime Patterns",
    x_axis_label="Crime 1",
    y_axis_label="Crime 2",
    width=700, height=400,
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# Dark theme
p.background_fill_color = "#252a34"
p.border_fill_color = "#252a34"
p.outline_line_color = None
p.title.text_color = "#eaeaea"
p.axis.axis_line_color = "#eaeaea"
p.axis.major_label_text_color = "#eaeaea"
p.axis.major_tick_line_color = "#eaeaea"
p.axis.minor_tick_line_color = "#eaeaea"
p.xaxis.axis_label_text_color = "#eaeaea"
p.yaxis.axis_label_text_color = "#eaeaea"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Scatter plot
p.scatter(
    x="x", y="y", source=source,
    size=7, color="#00b8d4", alpha=1.5, line_color=None
)

# Hover tool
hover = HoverTool(tooltips=[
    ("Day", "@day"),
    ("Time", "@time"),
    ("Crime 1 Count", "@x"),
    ("Crime 2 Count", "@y")
])
p.add_tools(hover)

# Dropdowns
select_crime1 = Select(title="Select Crime 1:", value=init_crime1, options=crime_list)
select_crime2 = Select(title="Select Crime 2:", value=init_crime2, options=crime_list)
select_crime1.styles = {"color": "white"}
select_crime2.styles = {"color": "white"}

# Convert mapping to JSON-safe format
hour_map_json = json.dumps({int(k): v for k, v in hour_to_day_and_time.items()})

# Callback
callback_code = """
var c1 = select_crime1.value;
var c2 = select_crime2.value;

var hoursAll = unfiltered_data['hour_of_week'];
var arrX = unfiltered_data[c1];
var arrY = unfiltered_data[c2];
var hour_map = JSON.parse(hour_map_json);

var new_hour = [];
var new_x = [];
var new_y = [];
var new_day = [];
var new_time = [];

for (var i = 0; i < hoursAll.length; i++) {
    var hour = hoursAll[i];
    new_hour.push(hour);
    new_x.push(arrX[i]);
    new_y.push(arrY[i]);
    new_day.push(hour_map[hour][0]);
    new_time.push(hour_map[hour][1]);
}

source.data = {
    hour: new_hour,
    x: new_x,
    y: new_y,
    day: new_day,
    time: new_time
};
source.change.emit();
"""

callback = CustomJS(
    args=dict(
        source=source,
        unfiltered_data=unfiltered_data,
        select_crime1=select_crime1,
        select_crime2=select_crime2,
        hour_map_json=hour_map_json
    ),
    code=callback_code
)

select_crime1.js_on_change("value", callback)
select_crime2.js_on_change("value", callback)

layout1 = column(row(select_crime1, select_crime2), p)

# Save to file
output_file("/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/thomasroennebek.github.io/assets/images/interactive/scatterplot.html")
save(layout1)

Loading BokehJS ...

'/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/thomasroennebek.github.io/assets/images/interactive/scatterplot.html'

# Time series

In [39]:
sf_crime_data = pd.read_csv(
    "/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/Data/Police_Incident_Reports_2003_2025_Cleaned.csv",
    low_memory=False
)

In [40]:
# Convert "Incident Date" to datetime
sf_crime_data["Incident Date"] = pd.to_datetime(sf_crime_data["Incident Date"], errors="coerce")
sf_crime_data.dropna(subset=["Incident Date"], inplace=True)

In [41]:
# Normalize categories
sf_crime_data["Incident Category"] = sf_crime_data["Incident Category"].replace({
    "Weapons Carrying Etc": "Weapon Laws",
    "Weapons Offense":      "Weapon Laws",
    "Weapons Offence":      "Weapon Laws",
    "Drug Violation":       "Drug/Narcotic",
    "Drug Offense":         "Drug/Narcotic",
    "Motor Vehicle Theft":  "Vehicle Theft",
    "Motor Vehicle Theft?": "Vehicle Theft",
    "Malicious Mischief":   "Vandalism"
})

In [42]:
focus_crime_data = sf_crime_data[sf_crime_data["Incident Category"].isin(focuscrimes)].copy()

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
    ColumnDataSource, CustomJS, Select, HoverTool,
    DatetimeTickFormatter, Range1d
)
from bokeh.layouts import column
import pandas as pd
import numpy as np
import datetime 
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import HoverTool, DatetimeTickFormatter
from bokeh.models import ColumnDataSource
from bokeh.models import Select, CustomJS


# Your filtered crime data already grouped by month and category
focus_crime_data["Month"] = focus_crime_data["Incident Date"].dt.to_period("M")
grouped = focus_crime_data.groupby(["Month", "Incident Category"]).size().reset_index(name="Count")

# Convert to datetime
grouped["Month"] = grouped["Month"].astype(str)
grouped["Month"] = pd.to_datetime(grouped["Month"], errors="coerce")

# Rename for clarity
grouped.rename(columns={"Month": "date", "Incident Category": "category", "Count": "count"}, inplace=True)

# Drop any missing
grouped.dropna(subset=["date", "category", "count"], inplace=True)

# Build "ALL CRIMES" monthly totals
all_crimes_df = focus_crime_data.copy()
all_crimes_df["Month"] = all_crimes_df["Incident Date"].dt.to_period("M")
all_grouped = all_crimes_df.groupby("Month").size().reset_index(name="count")
all_grouped["Month"] = all_grouped["Month"].astype(str)
all_grouped["date"] = pd.to_datetime(all_grouped["Month"], errors="coerce")
all_grouped["category"] = "ALL CRIMES"
all_grouped = all_grouped[["date", "count", "category"]]

# Combine with the main grouped data
grouped = pd.concat([grouped, all_grouped], ignore_index=True)

# Final structure:
# grouped = pd.DataFrame with columns: 'date', 'category', 'count'



source_all = ColumnDataSource(grouped)


# Start with just one category (e.g., "ALL CRIMES" or first in list)
init_category = "ALL CRIMES" if "ALL CRIMES" in grouped["category"].unique() else grouped["category"].iloc[0]
initial_df = grouped[grouped["category"] == init_category]

source_filtered = ColumnDataSource(initial_df)




# Unique list of crimes
crime_categories = grouped["category"].unique().tolist()
crime_select = Select(title="Select Crime:", value=init_category, options=sorted(crime_categories))

callback = CustomJS(args=dict(
    source_all=source_all,
    source_filtered=source_filtered,
    crime_select=crime_select
), code="""
    const selected = crime_select.value;
    const data_all = source_all.data;
    const data_filtered = source_filtered.data;

    const new_dates = [];
    const new_counts = [];
    const categories = data_all['category'];
    const dates = data_all['date'];
    const counts = data_all['count'];

    for (let i = 0; i < categories.length; i++) {
        if (categories[i] === selected) {
            new_dates.push(dates[i]);
            new_counts.push(counts[i]);
        }
    }

    data_filtered['date'] = new_dates;
    data_filtered['count'] = new_counts;
    source_filtered.change.emit();
""")

crime_select.js_on_change("value", callback)




p = figure(
    title="Monthly Crime Trends in San Francisco (2003–2025)",
    x_axis_type="datetime", width=800, height=400,
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# --- DARK THEME STYLING ---
p.background_fill_color = "#252a34"
p.border_fill_color = "#252a34"
p.outline_line_color = None
p.title.text_color = "#eaeaea"

p.axis.axis_line_color = "#eaeaea"
p.axis.major_label_text_color = "#eaeaea"
p.axis.major_tick_line_color = "#eaeaea"
p.axis.minor_tick_line_color = "#eaeaea"

p.xaxis.axis_label_text_color = "#eaeaea"
p.yaxis.axis_label_text_color = "#eaeaea"

p.grid.grid_line_color = "#444444"


# Styling (reuse your dark theme if you like!)
p.line(x='date', y='count', source=source_filtered, line_width=2, color="#00adb5")
#p.circle(x='date', y='count', source=source_filtered, size=6, color="#00adb5", alpha=0.8)

p.add_tools(HoverTool(
    tooltips=[("Month", "@date{%Y-%m}"), ("Count", "@count")],
    formatters={"@date": "datetime"}, mode='vline'
))

p.xaxis.axis_label = "Month"
p.yaxis.axis_label = "Number of Incidents"
p.xaxis.formatter = DatetimeTickFormatter(months="%Y-%m")

layout = column(crime_select, p)
show(layout)

RuntimeError: Models must be owned by only a single document, AllIndices(id='p2549', ...) is already in a doc

In [49]:
sf_crime_data.head()

,Incident Date,Incident Time,Incident Day of Week,Incident ID,Incident Number,Incident Code,Incident Category,Incident Description,Resolution,Police District,Analysis Neighborhood,Latitude,Longitude,Point
0,2023-03-01,05:02,Wednesday,1253795,236046151,6374,Larceny Theft,"theft, other property, >$950",open or active,mission,NaN,NaN,NaN,NaN
1,2023-03-13,10:59,Monday,1253551,230174885,7041,Recovered Vehicle,"vehicle, recovered, auto",open or active,out of sf,NaN,NaN,NaN,NaN
2,2023-03-14,18:44,Tuesday,1254024,230176728,7041,Recovered Vehicle,"vehicle, recovered, auto",open or active,out of sf,NaN,NaN,NaN,NaN
3,2023-02-15,03:00,Wednesday,1253786,236046123,6372,Larceny Theft,"theft, other property, $50-$200",open or active,mission,NaN,NaN,NaN,NaN
4,2023-03-13,11:26,Monday,1254195,236046850,6244,Larceny Theft,"theft, from locked vehicle, >$950",open or active,central,NaN,NaN,NaN,NaN


# MAP

In [9]:
# --- IMPORTS ---
import pandas as pd
import json
import copy
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
    GeoJSONDataSource, LinearColorMapper, ColorBar,
    HoverTool, Select, CustomJS, Div, LabelSet, ColumnDataSource
)
from bokeh.layouts import column, row
from shapely.geometry import shape, Point

output_notebook()

# --- LOAD & CLEAN DATA ---
sf_crime_data = pd.read_csv(
    "/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/Data/Police_Incident_Reports_2003_2025_Cleaned.csv",
    low_memory=False
)
sf_crime_data["Incident Date"] = pd.to_datetime(sf_crime_data["Incident Date"], errors="coerce")
sf_crime_data.dropna(subset=["Incident Date"], inplace=True)

sf_crime_data["Incident Hour"] = pd.to_datetime(
    sf_crime_data["Incident Time"], format="%H:%M", errors="coerce"
).dt.hour

sf_crime_data["Incident Category"] = sf_crime_data["Incident Category"].replace({
    "Weapons Carrying Etc": "Weapon Laws",
    "Weapons Offense":      "Weapon Laws",
    "Weapons Offence":      "Weapon Laws",
    "Drug Violation":       "Drug/Narcotic",
    "Drug Offense":         "Drug/Narcotic",
    "Motor Vehicle Theft":  "Vehicle Theft",
    "Motor Vehicle Theft?": "Vehicle Theft",
    "Malicious Mischief":   "Vandalism"
})

focuscrimes = {
    "Weapon Laws", "Prostitution", "Robbery", "Burglary",
    "Assault", "Drug/Narcotic", "Larceny Theft",
    "Vandalism", "Vehicle Theft", "Stolen Property"
}

focus_crime_data = sf_crime_data[sf_crime_data["Incident Category"].isin(focuscrimes)].copy()
focus_crime_data["Police District"] = focus_crime_data["Police District"].str.upper()
focus_crime_data = focus_crime_data[focus_crime_data["Police District"] != "OUT OF SF"]
focus_crime_data["Year"] = focus_crime_data["Incident Date"].dt.year.astype(str)  # Keep as string
focus_crime_data = focus_crime_data[focus_crime_data["Year"] >= "2015"]

# --- LOAD GEOJSON ---
with open("/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/Assignment 2/geojson.json", "r") as f:
    geo_data = json.load(f)

# --- PIVOT CRIME DATA ---
pivot = focus_crime_data.groupby(["Year", "Police District", "Incident Category"]).size().unstack(fill_value=0).reset_index()
pivot.columns.name = None
pivot["DISTRICT"] = pivot["Police District"]

all_years = sorted(pivot["Year"].unique())
all_categories = sorted([col for col in pivot.columns if col not in ["Year", "Police District", "DISTRICT"]])

data_dict = {cat: {} for cat in all_categories}
max_values = {}

for cat in all_categories:
    for y in all_years:
        subset = pivot[pivot["Year"] == y]
        data_dict[cat][y] = dict(zip(subset["DISTRICT"], subset[cat]))
    max_values[cat] = max(data_dict[cat][y].get(d, 0) for y in all_years for d in data_dict[cat][y])

# --- INITIAL SETUP ---
default_category = "All Crimes"  # Set default to "All Crimes"
default_year = all_years[0]

# --- ADD "All Crimes" to the Category Selector ---
all_categories_with_all = ["All Crimes"] + all_categories  # Add "All Crimes" at the start

category_select = Select(title="Select Crime Category", value=default_category, options=all_categories_with_all)
year_select = Select(title="Select Year", value=default_year, options=all_years)

# --- UPDATE DATA PREPARATION FOR "All Crimes" ---
def prepare_geojson(category, year):
    updated_geo = copy.deepcopy(geo_data)
    
    if category == "All Crimes":
        # Sum crime counts across all categories for each district
        for feature in updated_geo["features"]:
            district = feature["properties"]["DISTRICT"]
            total_count = sum(data_dict[cat][year].get(district, 0) for cat in all_categories)
            feature["properties"]["CRIME_COUNT"] = total_count
    else:
        # Use the existing category logic
        for feature in updated_geo["features"]:
            district = feature["properties"]["DISTRICT"]
            count = data_dict[category][year].get(district, 0)
            feature["properties"]["CRIME_COUNT"] = count
    
    return updated_geo

initial_geojson = prepare_geojson(default_category, default_year)
geo_source = GeoJSONDataSource(geojson=json.dumps(initial_geojson))
original_geo_json = json.dumps(copy.deepcopy(geo_data))

# Calculate max value for the color scale for "All Crimes"
max_all_crimes = {}
for year in all_years:
    max_all_crimes[year] = max(
        sum(data_dict[cat][year].get(district, 0) for cat in all_categories)
        for district in pivot["DISTRICT"]
    )

# --- CREATE HIGH-CONTRAST TURQUOISE-TO-BLUE COLOR MAPPER ---
color_mapper = LinearColorMapper(
    palette=["#00FEEF", "#09EBEE", "#19CEEB", "#28ACEA", "#388EE9", 
             "#3D76E0", "#0047A5", "#002F7B", "#001A53"], 
    low=0, 
    high=max_all_crimes[default_year]
)  # 9 distinct shades from bright turquoise to deep blue

# --- FIGURE ---
p = figure(tools="pan,wheel_zoom,reset,hover,save",
           x_axis_location=None, y_axis_location=None,
           width=850, height=500)
p.grid.grid_line_color = None  # Remove grid lines

# --- DARK THEME STYLING ---
p.background_fill_color = "#252a34"
p.border_fill_color = "#252a34"
p.outline_line_color = None
p.title.text_color = "#eaeaea"  # White title text

p.axis.axis_line_color = "#eaeaea"
p.axis.major_label_text_color = "#eaeaea"  # White axis labels
p.axis.major_tick_line_color = "#eaeaea"
p.axis.minor_tick_line_color = "#eaeaea"

p.xaxis.axis_label_text_color = "#eaeaea"  # White axis label
p.yaxis.axis_label_text_color = "#eaeaea"  # White axis label

# Plot the polygons with turquoise range coloring
p.patches('xs', 'ys', source=geo_source,
          fill_color={'field': 'CRIME_COUNT', 'transform': color_mapper},
          line_color="white", line_width=1, fill_alpha=0.5)  # Adjust fill_alpha for more transparency and white lines

title_div = Div(text=f"<h3>{default_category} Crimes in San Francisco – {default_year}</h3>", width=800)
title_div.styles = {"color": "white"}  # White title text color

# --- CALCULATE CENTROIDS AND ADJUST LABEL POSITION ---
centroids = []
for feature in geo_data["features"]:
    polygon = shape(feature["geometry"])
    centroid = polygon.centroid

    # Check if the district is "SOUTHERN" and adjust centroid if necessary
    if feature["properties"]["DISTRICT"] == "SOUTHERN":
        centroid = Point(centroid.x - 0.01, centroid.y - 0.015)  # Move more to the left and down slightly
    centroids.append((centroid.x, centroid.y))

# Create a data source for the labels (centroid x, centroid y, and district names)
district_names = [feature["properties"]["DISTRICT"] for feature in geo_data["features"]]
labels_data = ColumnDataSource(data=dict(
    x=[centroid[0] for centroid in centroids],
    y=[centroid[1] for centroid in centroids],
    name=district_names
))

# Add labels to the plot
labels = LabelSet(
    x='x', y='y', text='name', source=labels_data,
    x_offset=5, y_offset=5, text_align='center', text_baseline='middle',
    text_font_size="8px", text_color="white"
)

p.add_layout(labels)

# Hover tool
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("District", "@DISTRICT"), ("Crimes", "@CRIME_COUNT")]

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0, 0))
color_bar.background_fill_color = "#252a34"  # Set background to dark to match plot
color_bar.border_line_color = None
color_bar.major_label_text_color = "#eaeaea"  # White legend text color
p.add_layout(color_bar, 'right')

# --- CALLBACK UPDATE TO HANDLE "All Crimes" ---
callback_code = """
    const category = category_select.value;
    const year = year_select.value;  // ✅ use as string
    let yearData;

    if (category === "All Crimes") {
        yearData = {};
        // Sum all crime categories for each district
        for (const cat of Object.keys(data_dict)) {
            if (cat !== "All Crimes") {  // Skip the "All Crimes" category itself
                const catData = data_dict[cat][year];
                for (const district in catData) {
                    if (yearData[district]) {
                        yearData[district] += catData[district];
                    } else {
                        yearData[district] = catData[district];
                    }
                }
            }
        }
    } else {
        yearData = data_dict[category][year];
    }

    const geo = JSON.parse(original_geo_json);  // ✅ parse JSON string
    for (let i = 0; i < geo.features.length; i++) {
        const district = geo.features[i].properties.DISTRICT;
        geo.features[i].properties.CRIME_COUNT = yearData[district] || 0;
    }

    source.geojson = JSON.stringify(geo);
    color_mapper.high = category === "All Crimes" ? max_all_crimes[year] : max_values[category];
    title_div.text = `<h3>${category} Crimes in San Francisco – ${year}</h3>`;
"""

# --- CREATE CALLBACK ARGUMENTS ---
callback_args = dict(
    source=geo_source,
    original_geo_json=original_geo_json,
    data_dict=data_dict,
    max_values=max_values,
    color_mapper=color_mapper,
    title_div=title_div,
    category_select=category_select,
    year_select=year_select,
    max_all_crimes=max_all_crimes
)

category_callback = CustomJS(args=callback_args, code=callback_code)
year_callback = CustomJS(args=callback_args, code=callback_code)

category_select.js_on_change("value", category_callback)
year_select.js_on_change("value", year_callback)

# --- APPLY WHITE TEXT FOR DROPDOWN TITLES (USING STYLES) ---
category_select.styles = {"color": "white"}
year_select.styles = {"color": "white"}

# --- DISPLAY ---
show(column(row(category_select, year_select), title_div, p)) 

Loading BokehJS ...

In [10]:
from bokeh.io import output_file

# Specify the file path where you want to save the HTML
output_path = "/Users/Thomas/Desktop/Skole/Business Analytics/Social data analysis and visualization/thomasroennebek.github.io/assets/images/sf_crime_map.html"

# Use output_file to specify the HTML output file
output_file(output_path)

# --- DISPLAY ---
show(column(row(category_select, year_select), title_div, p))